检测结果

In [7]:
import os
import torch
from utils.general import non_max_suppression, scale_coords
from utils.datasets import letterbox
from utils.torch_utils import select_device
import cv2
import numpy as np

def detect_and_save_results(model, img_path, save_dir, img_size=640, conf_thres=0.25, iou_thres=0.45, device=''):
    # Load image
    img0 = cv2.imread(img_path)  # BGR
    assert img0 is not None, 'Image Not Found ' + img_path

    # Padded resize
    img = letterbox(img0, img_size, stride=32)[0]

    # Convert
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)

    # Load model
    if isinstance(device, str):
        device = select_device(device)
    model.to(device).eval()

    # Run inference
    img = torch.from_numpy(img).to(device)
    img = img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = model(img)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes=None, agnostic=False)

    # Process detections
    detections = []
    if len(pred):
        det = pred[0]
        if det is not None and len(det):
            # Rescale boxes from img_size to img0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

            # Create save directory if it does not exist
            os.makedirs(save_dir, exist_ok=True)

            # Save detections to a text file
            save_path = os.path.join(save_dir, os.path.splitext(os.path.basename(img_path))[0] + '.txt')
            with open(save_path, 'w') as f:
                for *xyxy, conf, cls in det:
                    xyxy = [coord.item() for coord in xyxy]
                    conf = conf.item()
                    cls = int(cls.item())
                    f.write(f"{cls} {conf:.4f} " + " ".join([str(int(x)) for x in xyxy]) + '\n')
                    detections.append({'class': cls, 'confidence': conf, 'bbox': xyxy})
        else:
            # Save an empty txt file if no detections
            save_path = os.path.join(save_dir, os.path.splitext(os.path.basename(img_path))[0] + '.txt')
            open(save_path, 'a').close()
            print(f"No detections for {img_path}, saved empty txt file.")

    return detections

if __name__ == "__main__":
    weights = 'runs/train/exp12/weights/best.pt'  # path to the model weights
    test_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\test_images'  # path to the directory containing test images
    save_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\test_results'  # directory to save detection results
    device = '0'  # GPU device

    # Load model
    device = select_device(device)
    model = torch.load(weights, map_location=device)['model'].float()  # load to FP32
    model.to(device).eval()

    # Process each image in the test directory
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        if img_path.endswith('.jpg') or img_path.endswith('.png'):
            detections = detect_and_save_results(model, img_path, save_dir, device=device)
            print(f"Processed {img_path}, detections: {len(detections)}")


Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_images\000021.jpg, detections: 47
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_images\001481.jpg, detections: 4
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_images\001484.jpg, detections: 8
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_images\001485.jpg, detections: 7
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_images\001486.jpg, detections: 6
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_images\001488.jpg, detections: 4
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_images\001489.jpg, detections: 6
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_images\001492.jpg, detections: 13
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_images\001495.jpg, detections: 8
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_images\001499.jpg, detections: 5


数据清理（同一个目标被检测成多个类别，只保留置信度最高的）

In [8]:
import os

# Function to read detections from .txt file
def read_detections_from_txt(txt_path):
    detections = []
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 6:
                category = int(parts[0])
                confidence = float(parts[1])
                bbox = list(map(float, parts[2:6]))
                detections.append((category, confidence, bbox))
    return detections

# Function to save detections to .txt file
def save_detections_to_txt(detections, txt_path):
    with open(txt_path, 'w') as f:
        for det in detections:
            category, confidence, bbox = det
            f.write(f"{category} {confidence:.4f} " + " ".join(map(str, map(int, bbox))) + '\n')

# Function to check if two bounding boxes have coordinates difference less than a threshold
def bbox_coordinate_close(bbox1, bbox2, threshold=100):
    for i in range(4):
        if abs(bbox1[i] - bbox2[i]) > threshold:
            return False
    return True

# Function to clean detection results
def clean_detections(results_dir, threshold=100):
    for txt_file in os.listdir(results_dir):
        if txt_file.endswith('.txt'):
            txt_path = os.path.join(results_dir, txt_file)
            detections = read_detections_from_txt(txt_path)
            
            # Initialize a list to keep the cleaned detections
            cleaned_detections = []
            
            # Initialize a list to keep track of which detections have been considered
            considered = [False] * len(detections)
            
            for i in range(len(detections)):
                if not considered[i]:
                    det1 = detections[i]
                    best_det = det1
                    for j in range(i + 1, len(detections)):
                        if not considered[j]:
                            det2 = detections[j]
                            if bbox_coordinate_close(det1[2], det2[2], threshold):
                                # Mark det2 as considered
                                considered[j] = True
                                # Keep the detection with the highest confidence
                                if det2[1] > best_det[1]:
                                    best_det = det2
                    # Mark det1 as considered
                    considered[i] = True
                    # Add the best detection to the cleaned list
                    cleaned_detections.append(best_det)
            
            # Save the cleaned detections back to the file
            save_detections_to_txt(cleaned_detections, txt_path)
            print(f"Processed {txt_path}")

if __name__ == "__main__":
    results_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\test_results'
    clean_detections(results_dir)


Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_results\000021.txt
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_results\001481.txt
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_results\001484.txt
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_results\001485.txt
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_results\001486.txt
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_results\001488.txt
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_results\001489.txt
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_results\001492.txt
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_results\001495.txt
Processed D:\Study\Ai\sklearn\Exercise_final\testimages\test_results\001499.txt


数据清理（置信度低于0.5的框删除）

In [9]:
import os

def filter_results(input_directory, confidence_threshold=0.5):
    # 遍历指定目录中的所有文件
    for filename in os.listdir(input_directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(input_directory, filename)
            filtered_data = []

            # 读取文件内容
            with open(file_path, 'r') as file:
                lines = file.readlines()

            # 过滤置信度低于阈值的数据
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 2:
                    category = parts[0]
                    try:
                        confidence = float(parts[1])
                        if confidence >= confidence_threshold:
                            filtered_data.append(line)
                    except ValueError:
                        # 如果置信度列不是浮点数，则跳过此行
                        continue

            # 将筛选后的数据写回原文件
            with open(file_path, 'w') as file:
                file.writelines(filtered_data)

if __name__ == "__main__":
    input_directory = r'D:\Study\Ai\sklearn\Exercise_final\testimages\test_results'
    filter_results(input_directory)


实际标签

In [11]:
import os
import xml.etree.ElementTree as ET

def convert_xml_to_txt(xml_dir, save_dir):
    # Create save directory if it does not exist
    os.makedirs(save_dir, exist_ok=True)

    # Define the class mapping
    class_mapping = {
        "holothurian": 0,
        "echinus": 1,
        "scallop": 2,
        "starfish": 3
    }

    for xml_file in os.listdir(xml_dir):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(xml_dir, xml_file)
            tree = ET.parse(xml_path)
            root = tree.getroot()
            
            txt_filename = os.path.splitext(xml_file)[0] + '.txt'
            txt_path = os.path.join(save_dir, txt_filename)
            
            with open(txt_path, 'w') as txt_file:
                for obj in root.iter('object'):
                    label = obj.find('name').text
                    if label not in class_mapping:
                        continue  # skip unknown classes
                    class_id = class_mapping[label]
                    
                    bbox = obj.find('bndbox')
                    x_min = int(bbox.find('xmin').text)
                    y_min = int(bbox.find('ymin').text)
                    x_max = int(bbox.find('xmax').text)
                    y_max = int(bbox.find('ymax').text)
                    
                    # Write to txt file in the required format: class_id 0 x_min y_min x_max y_max
                    txt_file.write(f"{class_id} 0 {x_min} {y_min} {x_max} {y_max}\n")

if __name__ == "__main__":
    xml_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\test_labels'  # path to the directory containing XML files
    save_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\test_labels_txt'  # directory to save converted TXT files

    convert_xml_to_txt(xml_dir, save_dir)


检出与实际对比

In [10]:
import os
import numpy as np

# Function to read detections from .txt file
def read_detections_from_txt(txt_path):
    detections = []
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                category = int(parts[0])
                confidence = float(parts[1])
                bbox = list(map(float, parts[2:6]))
                detections.append((category, confidence, bbox))
    return detections

# Function to check if two bounding boxes have coordinates difference less than a threshold
def bbox_coordinate_close(bbox1, bbox2, threshold=100):
    for i in range(4):
        if abs(bbox1[i] - bbox2[i]) > threshold:
            return False
    return True

# Function to compare detections with ground truth labels and count different situations
def compare_detections_with_labels(det_path, label_path):
    detections = read_detections_from_txt(det_path)
    labels = read_detections_from_txt(label_path)

    matched_labels = np.zeros(len(labels), dtype=bool)
    detection_counts = {'检出': 0, '错检出': 0, '未检出': len(labels)}
    modified_detections = []

    for detection in detections:
        det_category, _, det_bbox = detection
        found_match = False

        for i, label in enumerate(labels):
            label_category, _, label_bbox = label
            if bbox_coordinate_close(det_bbox, label_bbox, threshold=100):
                if det_category == label_category:
                    matched_labels[i] = True
                    found_match = True
                    detection_counts['检出'] += 1
                    detection_counts['未检出'] -= 1
                    modified_detections.append((det_category, 1.0, det_bbox))  # 设置置信度为1
                    break
                elif not matched_labels[i]:
                    found_match = True
                    detection_counts['错检出'] += 1
                    detection_counts['未检出'] -= 1
                    modified_detections.append((det_category, 0.0, det_bbox))  # 设置置信度为0

        if not found_match:
            detection_counts['错检出'] += 1
            modified_detections.append((det_category, 0.0, det_bbox))  # 设置置信度为0

    # 对于未检出的标签，添加到 modified_detections 列表中
    for i, matched in enumerate(matched_labels):
        if not matched:
            label_category, _, label_bbox = labels[i]
            modified_detections.append((label_category, 0.0, label_bbox))  # 设置置信度为0

    return detection_counts, modified_detections

# Function to save detections and counts to .txt file
def save_detections_and_counts_to_txt(detections, detection_counts, txt_path):
    with open(txt_path, 'w') as f:
        f.write(f"检出: {detection_counts['检出']}\n")
        f.write(f"错检出: {detection_counts['错检出']}\n")
        f.write(f"未检出: {detection_counts['未检出']}\n\n")
        for det in detections:
            category, confidence, bbox = det
            f.write(f"{category} {confidence:.4f} " + " ".join(map(str, map(int, bbox))) + '\n')

# Function to process all images in a directory
def process_images(results_dir, labels_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_files = os.listdir(results_dir)

    for img_file in image_files:
        if img_file.endswith('.txt'):
            result_path = os.path.join(results_dir, img_file)
            label_path = os.path.join(labels_dir, img_file)

            if os.path.exists(result_path) and os.path.exists(label_path):
                detection_counts, modified_detections = compare_detections_with_labels(result_path, label_path)
                output_file = os.path.join(output_dir, img_file)
                save_detections_and_counts_to_txt(modified_detections, detection_counts, output_file)
                print(f"Processed {img_file}")
                print(f"检出: {detection_counts['检出']}, 错检出: {detection_counts['错检出']}, 未检出: {detection_counts['未检出']}")

if __name__ == "__main__":
    results_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\test_results'
    labels_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\test_labels_txt'
    output_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\results_summary'

    process_images(results_dir, labels_dir, output_dir)


Processed 000021.txt
检出: 24, 错检出: 2, 未检出: 6
Processed 001481.txt
检出: 4, 错检出: 0, 未检出: 1
Processed 001484.txt
检出: 4, 错检出: 1, 未检出: 0
Processed 001485.txt
检出: 7, 错检出: 0, 未检出: 2
Processed 001486.txt
检出: 4, 错检出: 2, 未检出: 1
Processed 001488.txt
检出: 3, 错检出: 1, 未检出: 0
Processed 001489.txt
检出: 2, 错检出: 2, 未检出: 5
Processed 001492.txt
检出: 8, 错检出: 2, 未检出: 1
Processed 001495.txt
检出: 5, 错检出: 2, 未检出: 0
Processed 001499.txt
检出: 5, 错检出: 0, 未检出: 1


统计

In [11]:
import os

def generate_summary_txt(summary_dir, output_file):
    detection_counts = {'检出': 0, '错检出': 0, '未检出': 0}

    if not os.path.exists(summary_dir):
        print(f"Error: Directory '{summary_dir}' does not exist.")
        return

    summary_files = os.listdir(summary_dir)
    num_images = 0

    with open(output_file, 'w') as f:
        for file_name in summary_files:
            file_path = os.path.join(summary_dir, file_name)
            if os.path.isfile(file_path) and file_name.endswith('.txt'):
                num_images += 1
                with open(file_path, 'r') as img_file:
                    lines = img_file.readlines()
                    for line in lines:
                        if line.startswith('检出'):
                            detection_counts['检出'] += int(line.split(':')[1].strip())
                        elif line.startswith('错检出'):
                            detection_counts['错检出'] += int(line.split(':')[1].strip())
                        elif line.startswith('未检出'):
                            detection_counts['未检出'] += int(line.split(':')[1].strip())

        f.write(f"检出总数: {detection_counts['检出']}\n")
        f.write(f"错检出总数: {detection_counts['错检出']}\n")
        f.write(f"未检出总数: {detection_counts['未检出']}\n")

if __name__ == "__main__":
    summary_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\results_summary'
    output_file = r'D:\Study\Ai\sklearn\Exercise_final\testimages\results_summary\summary.txt'

    generate_summary_txt(summary_dir, output_file)


实际数据总目标数

In [17]:
import os

def count_total_lines(txt_dir):
    total_lines = 0

    if not os.path.exists(txt_dir):
        print(f"Error: Directory '{txt_dir}' does not exist.")
        return

    txt_files = [f for f in os.listdir(txt_dir) if f.endswith('.txt')]

    for file_name in txt_files:
        file_path = os.path.join(txt_dir, file_name)
        with open(file_path, 'r') as f:
            lines = f.readlines()
            total_lines += len(lines)

    return total_lines

if __name__ == "__main__":
    txt_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\test_labels_txt'

    total_lines = count_total_lines(txt_dir)
    print(f"总行数: {total_lines}")


总行数: 85


各类别检测准确率（正确检出总数/所有检出总数）

In [12]:
import os

# Function to read detections from .txt file and extract relevant data
def read_detections_from_txt(txt_path):
    detections = []
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 6:  # Skip lines that are not detections
                category = int(parts[0])
                confidence = float(parts[1])
                detections.append((category, confidence))
    return detections

# Function to process all detection files in a directory and calculate proportions
def calculate_confidence_proportions(results_dir, output_file):
    # Define categories
    categories = {
        0: 'holothurian',
        1: 'echinus',
        2: 'scallop',
        3: 'starfish'
    }
    
    # Initialize counts for each category
    category_counts = {cat: {'total': 0, 'conf_1': 0} for cat in categories.values()}
    results = []

    image_files = os.listdir(results_dir)
    for img_file in image_files:
        if img_file.endswith('.txt'):
            result_path = os.path.join(results_dir, img_file)
            detections = read_detections_from_txt(result_path)

            for category, confidence in detections:
                if category in categories:
                    category_name = categories[category]
                    category_counts[category_name]['total'] += 1
                    if confidence == 1.0:
                        category_counts[category_name]['conf_1'] += 1

    # Calculate proportions and print/save results
    with open(output_file, 'w') as f:
        for category_name, counts in category_counts.items():
            total = counts['total']
            conf_1 = counts['conf_1']
            proportion = conf_1 / total if total > 0 else 0
            result = f"{category_name}: {proportion:.4f} ({conf_1}/{total})"
            results.append(result)
            print(result)
            f.write(result + '\n')

if __name__ == "__main__":
    results_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\results_summary'
    output_file = r'D:\Study\Ai\sklearn\Exercise_final\testimages\results_summary\confidence_proportions.txt'
    calculate_confidence_proportions(results_dir, output_file)


holothurian: 0.4286 (3/7)
echinus: 0.6316 (24/38)
scallop: 0.6429 (18/28)
starfish: 0.8400 (21/25)


各类别实际准确率（正确检出总数/实际总数）

In [13]:
import os

# Function to read detections from .txt file and extract relevant data
def read_detections_from_txt(txt_path):
    detections = []
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 6:  # Skip lines that are not detections
                category = int(parts[0])
                confidence = float(parts[1])
                detections.append((category, confidence))
    return detections

# Function to read labels from .txt file and count each category
def count_labels_from_txt(txt_path):
    label_counts = {0: 0, 1: 0, 2: 0, 3: 0}
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 1:  # Skip lines that do not have at least one element (category)
                category = int(parts[0])
                if category in label_counts:
                    label_counts[category] += 1
    return label_counts

# Function to process all detection files in a directory and calculate proportions
def calculate_confidence_proportions(results_dir, labels_dir, output_file):
    # Define categories
    categories = {
        0: 'holothurian',
        1: 'echinus',
        2: 'scallop',
        3: 'starfish'
    }
    
    # Initialize counts for each category
    category_counts = {cat: {'total': 0, 'conf_1': 0, 'actual': 0} for cat in categories.values()}
    results = []

    # Count actual labels
    label_files = os.listdir(labels_dir)
    for label_file in label_files:
        if label_file.endswith('.txt'):
            label_path = os.path.join(labels_dir, label_file)
            label_counts = count_labels_from_txt(label_path)
            for category, count in label_counts.items():
                if category in categories:
                    category_name = categories[category]
                    category_counts[category_name]['actual'] += count

    # Count detections
    image_files = os.listdir(results_dir)
    for img_file in image_files:
        if img_file.endswith('.txt'):
            result_path = os.path.join(results_dir, img_file)
            detections = read_detections_from_txt(result_path)

            for category, confidence in detections:
                if category in categories:
                    category_name = categories[category]
                    category_counts[category_name]['total'] += 1
                    if confidence == 1.0:
                        category_counts[category_name]['conf_1'] += 1

    # Calculate proportions and print/save results
    with open(output_file, 'w') as f:
        for category_name, counts in category_counts.items():
            actual = counts['actual']
            conf_1 = counts['conf_1']
            proportion = conf_1 / actual if actual > 0 else 0
            result = f"{category_name}: {proportion:.4f} ({conf_1}/{actual})"
            results.append(result)
            print(result)
            f.write(result + '\n')

if __name__ == "__main__":
    results_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\results_summary'
    labels_dir = r'D:\Study\Ai\sklearn\Exercise_final\testimages\test_labels_txt'
    output_file = r'D:\Study\Ai\sklearn\Exercise_final\testimages\actual_accuracy.txt'
    calculate_confidence_proportions(results_dir, labels_dir, output_file)


holothurian: 0.5000 (3/6)
echinus: 0.7742 (24/31)
scallop: 0.7200 (18/25)
starfish: 0.9130 (21/23)
